In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time

import data_utils
import model_utils_train
import model_utils_infer
import train_utils
import evaluation
importlib.reload(data_utils)
importlib.reload(model_utils_train)
importlib.reload(model_utils_infer)


importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn

from transformers import DistilBertPreTrainedModel
from transformers import DistilBertForQuestionAnswering
from transformers import DistilBertConfig, DistilBertTokenizer

### Load data

In [2]:
file_dict = {"train1.1": "./data/tiny_train-v1.1.json",
             "train2.0": "./data/tiny_train-v2.0.json", 
             "dev1.1": "./data/dev-v1.1.json"}
train_data_v1 = data_utils.prep_data(file_dict["train1.1"])
train_data_v2 = data_utils.prep_data(file_dict["train2.0"])
dev_data_v1 = data_utils.prep_data(file_dict["dev1.1"])

### Run inference with Huggingface pretrained BERT fine-tuned on SQuAD

In [100]:
importlib.reload(model_utils)
importlib.reload(train_utils)
m = model_utils.PreTrainedSQuAD()
result = train_utils.inference(train_data_v2, m)

inference time (6 paragraphs): 57.1 sec
evaluation result:
 OrderedDict([('exact', 73.33333333333333), ('f1', 88.20374794287838), ('total', 45), ('HasAns_exact', 73.33333333333333), ('HasAns_f1', 88.20374794287838), ('HasAns_total', 45)])


### Run inference with Huggingface pretrained DistillBERT fine-tuned on SQuAD

In [7]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(train_data_v1, m)

inference time (6 paragraphs): 0.6 sec
exact match: 68.51851851851852, f1 score: 84.66444049777384


In [8]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(dev_data_v1, m)

inference time (2067 paragraphs): 112.2 sec
exact match: 69.24314096499526, f1 score: 79.05754999708667


### Train and run simple DistilBERT model

In [6]:
importlib.reload(model_utils_train)
importlib.reload(train_utils)
m = model_utils_train.SimpleDistilBERT()

In [ ]:
importlib.reload(train_utils)
result = train_utils.train(train_data=train_data_v1, 
                           val_data=train_data_v1, 
                           model=m, patience=100, lr=1e-3)

Epoch: 0, LR: 0.001, Train Loss: 80.5989, Val Loss: 139.0005, Val f1 0.000, epoch time: 1.5s
Epoch: 1, LR: 0.001, Train Loss: 12.9786, Val Loss: 132.6728, Val f1 0.000, epoch time: 1.5s
Epoch: 2, LR: 0.001, Train Loss: 12.7889, Val Loss: 132.6098, Val f1 0.000, epoch time: 1.5s
Epoch: 3, LR: 0.001, Train Loss: 12.6274, Val Loss: 133.2948, Val f1 2.104, epoch time: 1.6s
Epoch: 4, LR: 0.001, Train Loss: 12.6685, Val Loss: 132.4457, Val f1 2.896, epoch time: 1.6s
Epoch: 5, LR: 0.001, Train Loss: 12.6625, Val Loss: 132.6158, Val f1 3.084, epoch time: 1.6s
Epoch: 6, LR: 0.001, Train Loss: 12.6776, Val Loss: 132.2205, Val f1 4.726, epoch time: 1.6s
Epoch: 7, LR: 0.001, Train Loss: 12.6286, Val Loss: 131.6328, Val f1 0.000, epoch time: 1.5s
Epoch: 8, LR: 0.001, Train Loss: 12.6055, Val Loss: 132.5540, Val f1 0.000, epoch time: 1.5s
Epoch: 9, LR: 0.001, Train Loss: 12.5610, Val Loss: 131.8964, Val f1 0.000, epoch time: 1.5s
Epoch: 10, LR: 0.001, Train Loss: 12.7081, Val Loss: 134.0461, Val f1 

In [44]:
starts.sum(axis=1)

tensor([1, 1, 1, 1, 1], device='cuda:0')

In [37]:
torch.zeros(5).long()

tensor([0, 0, 0, 0, 0])

In [22]:
for batch in train_data_v1:
    m(batch)

torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([16, 183])
torch.Size([16, 183])
torch.Size([10, 2])
torch.Size([10, 2])
torch.Size([10, 190])
torch.Size([10, 190])
torch.Size([10, 2])
torch.Size([10, 2])
torch.Size([10, 112])
torch.Size([10, 112])
torch.Size([8, 2])
torch.Size([8, 2])
torch.Size([8, 325])
torch.Size([8, 325])
torch.Size([5, 2])
torch.Size([5, 2])
torch.Size([5, 224])
torch.Size([5, 224])
torch.Size([5, 2])
torch.Size([5, 2])
torch.Size([5, 247])
torch.Size([5, 247])


In [31]:
for i in range(len(batch)):
    print(batch.q[i])
    print(batch.a[i])
    s = torch.argmax(starts[i]).item()
    e = torch.argmax(ends[i]).item()
    print(t.decode(inputs["input_ids"][i, s:e+1]))
    print(t.convert_tokens_to_string(t.convert_ids_to_tokens(inputs["input_ids"][i, s:e+1])))
    print()

What was von Braun's role in the army's rocket program during during World War II?
technical director of the ballistic missile program
technical director of the ballistic missile program
technical director of the ballistic missile program

What was the name of the first vehicle to reach outer space?
the Aggregate-4
the aggregate - 4
the aggregate - 4

During WWII, who was in charge of the German army's rocket program?
General Dornberger
general dornberger
general dornberger

What was the first object to enter space?
Aggregate-4 (A-4) rocket
aggregate - 4 ( a - 4 ) rocket
aggregate - 4 ( a - 4 ) rocket

When did the Aggregate-4 (A-4) rocket reach space?
1942 and 1943
1942 and 1943
1942 and 1943



In [120]:
start_logits, end_logits = _[:, 0], _[:, 1]

In [28]:
len(train_data_v1)

54

In [88]:
start_logits

tensor([-0.0338, -0.1953,  0.3170,  0.1470,  0.3645,  0.2151,  0.4525,  0.0216,
         0.1056,  0.0023,  0.5347,  0.3591,  0.1755,  0.0509, -0.1648, -0.1314,
        -0.0548,  0.5700,  0.3085,  0.2198,  0.2443,  0.2767, -0.0936,  0.4682,
        -0.2643,  0.1275, -0.4731, -0.3886, -0.3462, -0.0608,  0.0363, -0.1420,
         0.1483, -0.3176, -0.2976, -0.2344, -0.3339, -0.2073, -0.1009,  0.0186,
        -0.2375,  0.2723,  0.1106,  0.3088,  0.3516,  0.0856,  0.2452,  0.0366,
        -0.0228, -0.1730,  0.1252,  0.0356,  0.0581,  0.1838,  0.0967, -0.3358,
         0.1892, -0.7193, -0.3923, -0.1567, -0.1160, -0.2515, -0.2789, -0.2837,
        -0.0509,  0.0798, -0.0807,  0.2901,  0.1099,  0.0578,  0.2417,  0.1205,
         0.2458,  0.2456,  0.3944, -0.3274, -0.0044, -0.2566,  0.0513,  0.1809,
        -0.1168, -0.2245,  0.0556, -0.0087,  0.2180,  0.0539,  0.1345, -0.0422,
        -0.1499,  0.0030,  0.1015, -0.1616, -0.0196, -0.0209, -0.0829, -0.1064,
         0.0142, -0.1297,  0.0788,  0.39

In [61]:
_[0][0].shape

torch.Size([16, 183, 768])

In [65]:
m.prepare_data(batch)["input_ids"].shape

torch.Size([16, 183])

In [62]:
m

SimpleDistilBERT(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_f

In [145]:
batch.context[440: 460]

' 6th and 4th centuri'

In [141]:
t.decode(inputs["input_ids"][13, 103:104])

'4th'

In [142]:
t.decode(inputs["input_ids"][13, 100:110])

'the 6th and 4th centuries bce. [ note 1'

In [143]:
batch.a_index[13]

449